# Importamos modelo y prepara datos

In [120]:
!pip install tensorflow -q
!pip install tf-keras -q

In [121]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import json

In [122]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PATH = "/content/drive/MyDrive/Semestre 7/Colabs/Benji/Move Predictor/"

df = pd.read_csv(DRIVE_PATH + "pokemons_vgc_normalized.csv")

model = load_model(DRIVE_PATH + "move_pred_model.keras")

with open(DRIVE_PATH + "pokemon2idx.json", "r") as f:
    pokemon2idx = json.load(f)

with open(DRIVE_PATH + "idx2pokemon.json", "r") as f:
    idx2pokemon = json.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [123]:
INPUT_COLS = ["j1_1","j1_2","j1_3","j1_4","j1_5","j1_6",
              "j2_1","j2_2","j2_3","j2_4","j2_5","j2_6",]
LABEL_COLS = ["j2_l1","j2_l2"]

X_subset = df[INPUT_COLS]
y_subset = df[LABEL_COLS].to_numpy()

In [124]:
model.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_13      │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_6         │ (None, 12, 64)    │     36,736 │ input_layer_13[0… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_12          │ (None, 768)       │          0 │ embedding_6[0][0] │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_35 (Dense)    │ (None, 64)        │     49,216 │ flatten_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_36 (Dense)    │ (None, 64)        │      4,160 │ dense_35[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lead1 (Dense)       │ (None, 574)       │     37,310 │ dense_36[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lead2 (Dense)       │ (None, 574)       │     37,310 │ dense_36[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 494,198 (1.89 MB)

 Trainable params: 164,732 (643.48 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 329,466 (1.26 MB)

In [125]:
def encode_X(df, pokemon2idx):
    return df.applymap(lambda p: pokemon2idx[p])

X = encode_X(X_subset, pokemon2idx).to_numpy()
Y = y_subset

/tmp/ipython-input-2742113699.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df.applymap(lambda p: pokemon2idx[p])


In [128]:
def test_rand_sample(model, X, Y, idx2pokemon):

    i = np.random.randint(0, len(X))

    X_sample = X[i:i+1]
    valid_options = X_sample[0][6:]
    valid_to_name = [idx2pokemon[str(int(p))] for p in valid_options]

    pred1, pred2 = model.predict(X_sample, verbose=0)
    logits1 = pred1[0]
    logits2 = pred2[0]

    best1 = max(valid_options, key=lambda pid: logits1[int(pid)])
    best2 = max(valid_options, key=lambda pid: logits2[int(pid)])

    p1 = str(best1)
    p2 = str(best2)

    true1 = Y[i][0]
    true2 = Y[i][1]

    print("Muestra aleatoria:", i)
    print("Pokémon del rival:")
    print(valid_to_name)
    print()
    print("Valores reales:")
    print("   - Lead 1:", true1)
    print("   - Lead 2:", true2)
    print()
    print("Predicción del modelo:")
    print("   - Pred 1:", idx2pokemon[p1])
    print("   - Pred 2:", idx2pokemon[p2])



# Probar el modelo

In [148]:
test_rand_sample(model, X, Y, idx2pokemon)

Muestra aleatoria: 3938
Pokémon del rival:
['amoonguss', 'fluttermane', 'incineroar', 'koraidon', 'lunala', 'ursaluna']

Valores reales:
   - Lead 1: incineroar
   - Lead 2: lunala

Predicción del modelo:
   - Pred 1: incineroar
   - Pred 2: lunala
